# Implémentation de méthodes élémentaires pour la classification supervisée : Naive Bayes et classifieur par plus proches voisins

Pour ce TP, nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix

Le jeu de données [Vertebral Column](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column) permet d'étudier les pathologies d'hernie discale et de Spondylolisthesis. Ces deux pathologies sont regroupées dans le jeu de données en une seule catégorie dite `Abnormale`. 

Il s'agit donc d'un problème de classification supervisée à deux classes :
- Normale (NO) 
- Abnormale (AB)    

avec 6 variables bio-mécaniques disponibles (features).

L'objectif du TP est d'implémenter quelques méthodes simples de classification supervisée pour ce problème.

# Importation des données

> Télécharger le fichier column_2C.dat depuis le site de l'UCI à [cette adresse](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column). 
>
> On peut importer les données sous python par exemple avec la librairie [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html). Vous pourrez au besoin consulter la documentation de la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). 
> 
> Le chemin donné dans la fonction `read_csv`est une chaîne de caractère qui spécifie le chemin complet vers le ficher sur votre machine. On peut aussi donner une adresse url si le fichier est disponible en ligne.
>
> Attention à la syntaxe pour les chemins sous Windows doit etre de la forme  `C:/truc/machin.csv`. 
> 
> Voir ce [blog](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) pour en savoir plus sur la "manipulation des chemins" sur des OS variés. 

In [2]:
file_path= "/home/rlucas/Documents/STASC/column_2C.dat"
Vertebral = pd.read_csv(file_path,
                          delim_whitespace= " ",
                          header= None)
Vertebral.columns = ["pelvic_incidence",
                                       "pelvic_tilt",
                                       "lumbar_lordosis_angle",
                                       "sacral_slope",
                                       "pelvic_radius",
                                       "degree_spondylolisthesis",
                                       "class"]

> Vérifier à l'aide des méthodes `.head()`  et `describe()` que les données sont bien importées.

In [3]:
print(Vertebral.head())
print("##########################")
print(Vertebral.describe())

   pelvic_incidence  pelvic_tilt  lumbar_lordosis_angle  sacral_slope  \
0             63.03        22.55                  39.61         40.48   
1             39.06        10.06                  25.02         29.00   
2             68.83        22.22                  50.09         46.61   
3             69.30        24.65                  44.31         44.64   
4             49.71         9.65                  28.32         40.06   

   pelvic_radius  degree_spondylolisthesis class  
0          98.67                     -0.25    AB  
1         114.41                      4.56    AB  
2         105.99                     -3.53    AB  
3         101.87                     11.21    AB  
4         108.17                      7.92    AB  
##########################
       pelvic_incidence  pelvic_tilt  lumbar_lordosis_angle  sacral_slope  \
count        310.000000   310.000000             310.000000    310.000000   
mean          60.496484    17.542903              51.930710     42.953871 

> Les librairies de Machine Learning telles que `sckitlearn` prennent en entrée des tableau numpy (pas des objets pandas). Créer un tableau numpy que vous nommerez `VertebralVar` pour les features et un vecteur numpy `VertebralClas` pour la variable de classe. Voir par exemple [ici](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy).

In [4]:
VertebralVar  = Vertebral.iloc[:, range(6)].to_numpy(copy=True)
VertebralClas = Vertebral["class"].to_numpy(copy = True)
print(VertebralVar)
print(VertebralClas)

[[ 63.03  22.55  39.61  40.48  98.67  -0.25]
 [ 39.06  10.06  25.02  29.   114.41   4.56]
 [ 68.83  22.22  50.09  46.61 105.99  -3.53]
 ...
 [ 61.45  22.69  46.17  38.75 125.67  -2.71]
 [ 45.25   8.69  41.58  36.56 118.55   0.21]
 [ 33.84   5.07  36.64  28.77 123.95  -0.2 ]]
['AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB

# Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 60% des observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de test. 

In [5]:
from sklearn.model_selection import train_test_split
VertebralVar_train,VertebralVar_test,VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar, VertebralClas)
ntot = len(VertebralClas)
ntrain = len(VertebralClas_train)
ntest = len(VertebralClas_test)
print(ntot, ntrain, ntest)

310 232 78


Remarque : on peut aussi le faire à la main avec la fonction [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

# Extraction des deux classes

> Extraire les deux sous-échantillons de classes respectives "Abnormale" et "Normale" pour les données d'apprentissage et de test.

In [6]:
print(len(VertebralClas_train), len(VertebralVar_train))
VertebralVar_train_AB = np.array([VertebralVar_train[i] for i in [j for j in range(len(VertebralClas_train)) if VertebralClas_train[j] == "AB"]])
VertebralVar_train_NO = np.array([VertebralVar_train[i] for i in [j for j in range(len(VertebralClas_train)) if VertebralClas_train[j] == "NO"]])
print(VertebralClas_train)

232 232
['AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB'
 'NO' 'AB' 'NO' 'AB' 'AB' 'NO' 'NO' 'NO' 'NO' 'AB' 'AB' 'AB' 'AB' 'NO'
 'NO' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'NO' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'NO' 'NO' 'AB'
 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO'
 'AB' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'NO' 'NO'
 'AB' 'AB' 'NO' 'NO' 'NO' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB'
 'NO' 'NO' 'NO' 'NO' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'AB'
 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO'
 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'NO' 'AB'
 'NO' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'NO' 'AB' 'AB' 'NO' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB

In [7]:
n_AB = len(VertebralVar_train_AB)
n_NO = len(VertebralVar_train_NO)
print(n_AB)
print(n_NO)

161
71


# Gaussian Naive Bayes

Nous allons ajuster un classifieur naif bayesien sur les données d'apprentissage.

Pour une observation $x \in \mathbb R^6$, la régle du MAP consiste à choisir la catégorie $\hat y (x) = \hat k $ qui maximise (en $k$) 
$$ score_k(x) = \hat \pi_k \prod_{j=1} ^6  \hat f_{k,j}(x_j)   $$
où :
- $k$ est le numéro de la classe ;
- $\hat \pi_k$ est la proportion observée de la classe $k$, 
- $\hat f_{k,j} $ est la densité gaussienne univariée de la classe $k$ pour la variable $j$. Les paramètres de cette loi valent (ajustés par maximum de vraisemblance) :
    - $\hat \mu_{k,j}$ : la moyenne empirique de la variable $X^j$ restreinte à la classe k,
    - $ \hat \sigma^2_{k,j}$ : la variance empirique de la variable $X^j$ restreinte à la classe k.
    
Noter que la fonction $x \mapsto  \prod_{j=1} ^6  f_{k,j}(x_j) $ peut aussi être vue comme une densité gaussienne multidimensionnelle de moyenne $(\mu_{k,1}, \dots, \mu_{k,6})$ et de matrice de covariance diagonale $diag(\hat \sigma^2_{k,1},\dots,\hat  \sigma^2_{k,6})$. Cette remarque évite de devoir calculer le produit de 6 densités univariées, à la place on calcule plus directement la valeur de la densité multidimensionnelle.

Pour calculer la valeur de la densité d'une gaussienne multidimensionnelle en un point $x$ de $\mathbb R ^d$ on peut utililser la fonction [`multivariate_normal`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) de la librairie [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html). 

On pourra utiliser la fonction `var` de numpy pour calculer le vecteur des variances.

Calcul des moyennes et des variances de chaque variable pour chacun des deux groupes :

In [8]:
mean_AB = np.average(VertebralVar_train_AB, axis = 0)
mean_NO = np.average(VertebralVar_train_NO, axis = 0)

# variances estimées variable par variable pour AB (sur le train) :
var_AB = np.var(VertebralVar_train_AB, axis = 0)
# variances estimées variable par variable pour NO (sur le train) :
var_NO = np.var(VertebralVar_train_NO, axis = 0)

# on forme les matrices de covariance (matrices diagonales car indep) :
Cov_NB_AB = np.diag(var_AB) 
Cov_NB_NO =  np.diag(var_NO) 

Calcul du "score" sur chaque groupe pour chaque element des données test : 

In [9]:
score_NB_test = np.array([(n_AB/(n_AB + n_NO) * multivariate_normal.pdf(x, mean_AB, Cov_NB_AB), n_NO/(n_AB + n_NO) * multivariate_normal.pdf(x, mean_NO, Cov_NB_NO)) for x in VertebralVar_test])
pred_NB_test = [("AB" if score[0] > score[1] else "NO") for score in score_NB_test]
print(pred_NB_test)
print(VertebralClas_test)


['AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'AB', 'NO', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'NO', 'AB', 'NO', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'AB', 'NO', 'AB', 'NO', 'AB', 'NO', 'AB', 'NO', 'AB', 'NO', 'AB']
['AB' 'AB' 'AB' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO'
 'AB' 'NO' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB'
 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'NO' 'AB' 'NO'
 'AB' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB'
 'NO' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'NO' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB'
 'NO' 'AB' 'NO' 'AB' 'NO' 'AB' 'NO' 'AB']


La matrice de confusion est une matrice qui synthétise les performances d'une régle de classification. Chaque ligne correspond à une classe réelle, chaque colonne correspond à une classe estimée. La cellule (ligne L, colonne C) contient le nombre d'éléments de la classe réelle L qui ont été estimés comme appartenant à la classe C. Voir par exemple [ici](https://fr.wikipedia.org/wiki/Matrice_de_confusion).

> Evaluer les performances de la méthode sur l'échantillon test. Vous pourrez utiliser la fonction [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) de la librairie [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [10]:
cnf_matrix_NB_test = confusion_matrix(y_true = VertebralClas_test, y_pred = pred_NB_test)
cnf_matrix_NB_test.astype('float') / cnf_matrix_NB_test.sum(axis=1).reshape(-1,1) 

array([[0.67346939, 0.32653061],
       [0.10344828, 0.89655172]])

>  Il existe bien sûr une fonction scikit-learn  pour la méthode Naive Bayes : voir [ici](http://scikit-learn.org/stable/modules/naive_bayes.html). Vérifier que votre prédicteur donne la même réponse de cette fonction.

In [11]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(VertebralVar_train, VertebralClas_train)
predicted = gnb.predict(VertebralVar_test)
cnf2_matrix_NB_test = confusion_matrix(y_true = VertebralClas_test, y_pred = predicted)
cnf2_matrix_NB_test.astype('float') / cnf2_matrix_NB_test.sum(axis=1).reshape(-1,1) 

array([[0.67346939, 0.32653061],
       [0.10344828, 0.89655172]])

# Classifieur par plus proches voisins

Il est préférable d'utiliser la structure de données de type [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) pour effectuer des requêtes de plus proches voisins dans un nuage de points. 

> Contruction du k-d tree pour les données train (pour la métrique euclidienne) :

In [12]:
from sklearn.neighbors import KDTree
tree =  KDTree(VertebralVar_train,6)

> Rechercher les 10 plus proches voisins dans les données d'apprentissage du premier point des données de test et afficher les classes de ces observations voisines.

In [13]:
indices_voisins =  tree.query(VertebralVar_train, 10, return_distance = False)
print(indices_voisins)
classes_voisins = VertebralClas_train[indices_voisins]
print(classes_voisins)    

[[  0  94 127 ...  37  39 148]
 [  1 108  85 ... 191  62  27]
 [  2 139  34 ...  89 140 147]
 ...
 [229   9 117 ...  33 121  13]
 [230 102 200 ... 131  76  74]
 [231  51 224 ...  37 206  74]]
[['AB' 'NO' 'AB' ... 'AB' 'AB' 'AB']
 ['AB' 'AB' 'AB' ... 'AB' 'AB' 'NO']
 ['AB' 'AB' 'AB' ... 'AB' 'NO' 'AB']
 ...
 ['NO' 'NO' 'AB' ... 'AB' 'AB' 'AB']
 ['NO' 'NO' 'NO' ... 'NO' 'NO' 'NO']
 ['NO' 'NO' 'NO' ... 'AB' 'NO' 'NO']]


Pour le classifieur par plus proches vosins, la prediction est la classe majoritaire des k plus proches voisins.

> Donner la prédiction pour le premier point de test par vote majoritaire sur ses 10 plus proches voisins 

In [14]:
def voisinage(X, k):
    indices_voisins =  tree.query([X], k, return_distance = False)
    #print(indices_voisins)
    classes_voisins = VertebralClas_train[indices_voisins]
    #print(classes_voisins)
    if np.count_nonzero(classes_voisins[0] == "AB") < np.count_nonzero(classes_voisins[0] == "NO"):
        return "NO"  
    else:
        return "AB"

voisinage(VertebralVar_test[0], 10)

'AB'

> Donner la prediction du classifieur ppv pour toutes les données de test. Evaluer la qualité du classifieur.

In [15]:
k_class = 11  #nombre de plus proche voisins utilisés
pred_kNN_test =  [voisinage(X, k_class) for X in VertebralVar_test]
cnf_matrix_kNN = confusion_matrix(y_true = VertebralClas_test, y_pred = pred_kNN_test)
cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1).reshape(-1,1) 

array([[0.81632653, 0.18367347],
       [0.24137931, 0.75862069]])

Il existe bien sûr une fonction scikit-learn pour le classifieur plus proche voisin, voir [ici](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).